# Introduction

## LLMs and Chat Models

In [ ]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI, ChatAnthropic
 
# llm = OpenAI(): deprecatetd
chat = ChatOpenAI()

# a = llm.predict("How many planets are there?"): deprecated
b = chat.predict("How many planets are there?")
 
b

'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

## Predict Messages

In [2]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1,
)

In [3]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in Italian.",
    ),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(
        content="What is the distance between Mexico and Thailand. Also, what is your name?",
    ),
]

chat.predict_messages(messages)

AIMessage(content='La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come posso aiutarti oggi?')

## Prompt Templates

In [ ]:
from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?")

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line.'

In [5]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

## OutputParser and LCEL

In [6]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    

p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [7]:
template =  ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}"),
])

prompt = template.format_messages(
    max_items=10,
    question="What are the colors?",
)

result = chat.predict_messages(prompt)

p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [8]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?",
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

## Chaining Chains

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

In [10]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on makin traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "Indian",
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Chicken Tikka Masala

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) tomato sauce
- 1 cup heavy cream
- 2 tsp garam masala
- 1 tsp ground cumin
- 1 tsp ground coriander
- 1 tsp paprika
- 1/2 tsp turmeric
- Salt and pepper, to taste
- Fresh cilantro, for garnish

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tbsp vegetable oil, 1 tsp garam masala, 1/2 tsp cumin, 1/2 tsp coriander, paprika, turmeric, salt, and pepper. Add the chicken pieces and marinate for at least 1 hour, preferably overnight.
2. Preheat the oven to 400°F (200°C). Thread the marinated chicken onto skewers and pl

AIMessageChunk(content="To make this Chicken Tikka Masala recipe vegetarian, we can replace the chicken with a suitable alternative. One popular substitute for chicken in Indian dishes is paneer, which is a type of Indian cottage cheese. Here's how you can modify the recipe:\n\n**Vegetarian Chicken Tikka Masala**\n\n**Ingredients:**\n- 1 lb paneer, cut into bite-sized pieces\n- 1 cup plain yogurt\n- 2 tbsp lemon juice\n- 2 tbsp vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) tomato sauce\n- 1 cup heavy cream\n- 2 tsp garam masala\n- 1 tsp ground cumin\n- 1 tsp ground coriander\n- 1 tsp paprika\n- 1/2 tsp turmeric\n- Salt and pepper, to taste\n- Fresh cilantro, for garnish\n\n**Instructions:**\n1. In a bowl, combine the yogurt, lemon juice, 1 tbsp vegetable oil, 1 tsp garam masala, 1/2 tsp cumin, 1/2 tsp coriander, paprika, turmeric, salt, and pepper. Add the paneer pieces and marinate for at least 1 hour, preferably

# Model IO

## FewShotPromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

# t = PromptTemplate(
#     template = "What is the capital of {country}?",
#     input_variables = ["country"],
# )

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Population: 67 million
        Language: French
        """,
    },
    {
        "question": "What do you know about Spain?",
        "answer": """
        Here is what I know:
        Capital: Madrid
        Population: 47 million
        """
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        Here is what I know:
        Capital: Rome
        Population: 60 million
        """
    }
]

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    suffix = "Human: What do you know about {country}?",
    input_variables = ["country"],
)

chain = prompt | chat

chain.invoke({
    "country": "Germany",
})

AI:
        Here is what I know:
        Capital: Berlin
        Population: 83 million
        Language: German

AIMessageChunk(content='AI:\n        Here is what I know:\n        Capital: Berlin\n        Population: 83 million\n        Language: German')

## FewShotChatMessagePromptTemplate

In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {question}?"),
    ("ai", "{answer}"),
])

examples = [
    {
        "question": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Population: 67 million
        Language: French
        """,
    },
    {
        "question": "Spain",
        "answer": """
        Here is what I know:
        Capital: Madrid
        Population: 47 million
        """
    },
    {
        "question": "Italy",
        "answer": """
        Here is what I know:
        Capital: Rome
        Population: 60 million
        """
    }
]

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert."),
    example_prompt,
    ("human", "What do you know about {country}?"),
])

chain = final_prompt | chat

chain.invoke({
    "country": "Germany",
})


        Here is what I know:
        Capital: Berlin
        Population: 83 million
        

AIMessageChunk(content='\n        Here is what I know:\n        Capital: Berlin\n        Population: 83 million\n        ')

In [24]:
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Population: 67 million
        Language: French
        """,
    },
    {
        "question": "What do you know about Spain?",
        "answer": """
        Here is what I know:
        Capital: Madrid
        Population: 47 million
        """
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        Here is what I know:
        Capital: Rome
        Population: 60 million
        """
    }
]

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt = example_prompt,
    max_length = 300,
)

random_example_selector = RandomExampleSelector(examples=examples)

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = random_example_selector,
    suffix = "Human: What do you know about {country}?",
    input_variables = ["country"],
)

prompt.format(country="France")

'Human:What do you know about Italy?\nAI:\n        Here is what I know:\n        Capital: Rome\n        Population: 60 million\n        \n\nHuman: What do you know about France?'

## Serialization and Composition

In [25]:
from langchain.prompts import load_prompt

prompt = load_prompt("prompt.json")

prompt.format(country="France")

'What is the capital of France?'

In [26]:
prompt = load_prompt("prompt.yaml")

prompt.format(country="France")

'What is the capital of France?'

In [32]:
from langchain.prompts.pipeline import PipelinePromptTemplate

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    final_prompt = final,
    pipeline_prompts = prompts,
)

full_prompt.format(
    character="Pirate",
    example_question="What is the meaning of life?",
    example_answer="The meaning of life is to find the treasure!",
    question="How old are you?",
)

chain = full_prompt | chat

chain.invoke({
    "character": "Pirate",
    "example_question": "What is the meaning of life?",
    "example_answer": "The meaning of life is to find the treasure!",
    "question": "How old are you?",
})

Arrr, me age be as old as the seas themselves! But a true pirate never reveals his exact age, matey.

AIMessageChunk(content='Arrr, me age be as old as the seas themselves! But a true pirate never reveals his exact age, matey.')

## Caching

In [35]:
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
set_debug(True)

chat = ChatOpenAI(temperature=0.1)

chat.predict("How do you make a cake?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make a cake?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [2.96s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Here is a basic recipe for making a simple vanilla cake:\n\nIngredients:\n- 2 1/2 cups all-purpose flour\n- 1 1/2 cups granulated sugar\n- 1 cup unsalted butter, softened\n- 4 large eggs\n- 1 cup milk\n- 2 teaspoons vanilla extract\n- 2 1/2 teaspoons baking powder\n- 1/2 teaspoon salt\n\nInstructions:\n1. Preheat your oven to 350°F (180°C) and grease and flour two 9-inch round cake pans.\n2. In a medium bowl, whisk together the flour, baking powder, and salt. Set aside.\n3. In a large mixing bowl, cream together the butter and sugar until light and fluffy.\n4. Add the eggs one at a time, mixing well after each addition.\n5. Stir in the vanilla extract.\n6. Gradually add the dry ingredients to the wet ingredients, alternating with the milk, beginning an

'Here is a basic recipe for making a simple vanilla cake:\n\nIngredients:\n- 2 1/2 cups all-purpose flour\n- 1 1/2 cups granulated sugar\n- 1 cup unsalted butter, softened\n- 4 large eggs\n- 1 cup milk\n- 2 teaspoons vanilla extract\n- 2 1/2 teaspoons baking powder\n- 1/2 teaspoon salt\n\nInstructions:\n1. Preheat your oven to 350°F (180°C) and grease and flour two 9-inch round cake pans.\n2. In a medium bowl, whisk together the flour, baking powder, and salt. Set aside.\n3. In a large mixing bowl, cream together the butter and sugar until light and fluffy.\n4. Add the eggs one at a time, mixing well after each addition.\n5. Stir in the vanilla extract.\n6. Gradually add the dry ingredients to the wet ingredients, alternating with the milk, beginning and ending with the dry ingredients. Mix until just combined.\n7. Divide the batter evenly between the prepared cake pans and smooth the tops with a spatula.\n8. Bake in the preheated oven for 25-30 minutes, or until a toothpick inserted i

In [36]:
chat.predict("How do you make a cake?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make a cake?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Here is a basic recipe for making a simple vanilla cake:\n\nIngredients:\n- 2 1/2 cups all-purpose flour\n- 1 1/2 cups granulated sugar\n- 1 cup unsalted butter, softened\n- 4 large eggs\n- 1 cup milk\n- 2 teaspoons vanilla extract\n- 2 1/2 teaspoons baking powder\n- 1/2 teaspoon salt\n\nInstructions:\n1. Preheat your oven to 350°F (180°C) and grease and flour two 9-inch round cake pans.\n2. In a medium bowl, whisk together the flour, baking powder, and salt. Set aside.\n3. In a large mixing bowl, cream together the butter and sugar until light and fluffy.\n4. Add the eggs one at a time, mixing well after each addition.\n5. Stir in the vanilla extract.\n6. Gradually add the dry ingredients to the wet ingredients, alternating with the milk, beginning and end

'Here is a basic recipe for making a simple vanilla cake:\n\nIngredients:\n- 2 1/2 cups all-purpose flour\n- 1 1/2 cups granulated sugar\n- 1 cup unsalted butter, softened\n- 4 large eggs\n- 1 cup milk\n- 2 teaspoons vanilla extract\n- 2 1/2 teaspoons baking powder\n- 1/2 teaspoon salt\n\nInstructions:\n1. Preheat your oven to 350°F (180°C) and grease and flour two 9-inch round cake pans.\n2. In a medium bowl, whisk together the flour, baking powder, and salt. Set aside.\n3. In a large mixing bowl, cream together the butter and sugar until light and fluffy.\n4. Add the eggs one at a time, mixing well after each addition.\n5. Stir in the vanilla extract.\n6. Gradually add the dry ingredients to the wet ingredients, alternating with the milk, beginning and ending with the dry ingredients. Mix until just combined.\n7. Divide the batter evenly between the prepared cake pans and smooth the tops with a spatula.\n8. Bake in the preheated oven for 25-30 minutes, or until a toothpick inserted i

In [38]:
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path="cache.db"))

chat.predict("How do you make a cake?")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make a cake?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [3.02s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Here is a basic recipe for making a simple vanilla cake:\n\nIngredients:\n- 2 1/2 cups all-purpose flour\n- 1 1/2 cups granulated sugar\n- 1 cup unsalted butter, softened\n- 4 large eggs\n- 1 cup milk\n- 2 teaspoons vanilla extract\n- 2 1/2 teaspoons baking powder\n- 1/2 teaspoon salt\n\nInstructions:\n1. Preheat your oven to 350°F (180°C) and grease and flour two 9-inch round cake pans.\n2. In a medium bowl, whisk together the flour, baking powder, and salt. Set aside.\n3. In a large mixing bowl, cream together the butter and sugar until light and fluffy.\n4. Add the eggs one at a time, mixing well after each addition.\n5. Stir in the vanilla extract.\n6. Gradually add the dry ingredients to the wet ingredients, alternating with the milk, beginning an

'Here is a basic recipe for making a simple vanilla cake:\n\nIngredients:\n- 2 1/2 cups all-purpose flour\n- 1 1/2 cups granulated sugar\n- 1 cup unsalted butter, softened\n- 4 large eggs\n- 1 cup milk\n- 2 teaspoons vanilla extract\n- 2 1/2 teaspoons baking powder\n- 1/2 teaspoon salt\n\nInstructions:\n1. Preheat your oven to 350°F (180°C) and grease and flour two 9-inch round cake pans.\n2. In a medium bowl, whisk together the flour, baking powder, and salt. Set aside.\n3. In a large mixing bowl, cream together the butter and sugar until light and fluffy.\n4. Add the eggs one at a time, mixing well after each addition.\n5. Stir in the vanilla extract.\n6. Gradually add the dry ingredients to the wet ingredients, alternating with the milk, beginning and ending with the dry ingredients. Mix until just combined.\n7. Divide the batter evenly between the prepared cake pans.\n8. Bake in the preheated oven for 25-30 minutes, or until a toothpick inserted into the center of the cakes comes o

## Serialization

In [39]:
from langchain.callbacks import get_openai_callback

set_llm_cache(None)

with get_openai_callback() as usage:
    chat.predict("How do you make a cake?")
    print(usage)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make a cake?"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [3.73s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Here is a basic recipe for making a simple vanilla cake:\n\nIngredients:\n- 2 1/2 cups all-purpose flour\n- 1 1/2 cups granulated sugar\n- 1 cup unsalted butter, softened\n- 4 large eggs\n- 1 cup milk\n- 2 teaspoons vanilla extract\n- 2 1/2 teaspoons baking powder\n- 1/2 teaspoon salt\n\nInstructions:\n1. Preheat your oven to 350°F (180°C) and grease and flour two 9-inch round cake pans.\n2. In a medium bowl, whisk together the flour, baking powder, and salt. Set aside.\n3. In a large mixing bowl, cream together the butter and sugar until light and fluffy.\n4. Add the eggs one at a time, mixing well after each addition.\n5. Stir in the vanilla extract.\n6. Gradually add the dry ingredients to the wet ingredients, alternating with the milk, beginning an

In [40]:
from langchain.llms.openai import OpenAI

chat = OpenAI(temperature=0.1, max_tokens=450, model="gpt-3.5-turbo-16k")

chat.save("model.json")

In [41]:
from langchain.llms.loading import load_llm

chat = load_llm("model.json")

chat

/Users/davidminjae/.local/share/virtualenvs/fullstack-gpt-Wpua4Cas/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/davidminjae/.local/share/virtualenvs/fullstack-gpt-Wpua4Cas/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-16k', model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})

# Memory

## ConversationBufferMemory

In [2]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "Hi!"}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

## ConversationBufferWindowMemory

In [3]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,
)

def add_messages(input, output):
    memory.save_context({"input": input}, {"output": output})

add_messages(1, 1)

In [4]:
add_messages(2, 2)
add_messages(3, 3)
add_messages(4, 4)

In [5]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [6]:
add_messages(5, 5)
memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

## ConversationSummaryMemory

In [7]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def get_history():
    return memory.load_memory_variables({})

add_messages("Hi I'm David, I live in South Korea.", "Wow that is so cool!")

In [8]:
add_messages("South Korea is so pretty", "I wish I could visit.")

In [9]:
get_history()

{'history': 'David introduces himself as living in South Korea. The AI responds by expressing admiration for his location, wishing it could visit because South Korea is so pretty.'}

## ConversationSummaryBufferMemory

In [14]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=50, return_messages=True)

add_messages("Hi I'm David, I live in South Korea.", "Wow that is so cool!")
add_messages("South Korea is so pretty", "I wish I could visit.")
add_messages("How far is South Korea from Japan?", "It is about 1000 km.")
add_messages("What is the capital of South Korea?", "Seoul")
get_history()

{'history': [SystemMessage(content='David introduces himself as living in South Korea, and the AI responds enthusiastically. The human mentions how pretty South Korea is, and the AI expresses a wish to visit.'),
  HumanMessage(content='How far is South Korea from Japan?'),
  AIMessage(content='It is about 1000 km.'),
  HumanMessage(content='What is the capital of South Korea?'),
  AIMessage(content='Seoul')]}

## ConversationKGMemory

In [15]:
from langchain.memory import ConversationKGMemory

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True
)

add_messages("Hi I'm David, I live in South Korea.", "Wow that is so cool!")

In [16]:
memory.load_memory_variables({"input": "who is David?"})

{'history': [SystemMessage(content='On David: David lives in South Korea.')]}

## Memory on LLMChain

In [21]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True
)

chain.predict(question="What is the capital of France?")



> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of France?

> Finished chain.


'The capital of France is Paris.'

In [22]:
memory.load_memory_variables({})

{'history': 'Human: What is the capital of France?\nAI: The capital of France is Paris.'}

In [23]:
template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human: {question}
    You:
"""

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history"
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True
)

chain.predict(question="What is the capital of France?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human: What is the capital of France?
    You:


> Finished chain.


'The capital of France is Paris.'

In [24]:
chain.predict(question="What did I ask?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: What is the capital of France?
AI: The capital of France is Paris.
    Human: What did I ask?
    You:


> Finished chain.


'You asked for the capital of France.'

## Chat Based Memory

In [25]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)

## LCEL Based Memory

In [29]:
from langchain.schema.runnable import RunnablePassthrough

def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({
        "question": question,
    })
    memory.save_context({"input": question}, {"output": result.content})
    return result

invoke_chain("What is the capital of France?")

AIMessage(content='The capital of France is Paris.')

# RAG

## Data Loaders and Splitters

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

# splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 200,
#     chunk_overlap = 50
# )

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader('./files/chapter_one.txt')

# docs = loader.load()
# docs.split_documents(splitter)

loader.load_and_split(text_splitter=splitter)

Created a chunk of size 963, which is longer than the specified 600
Created a chunk of size 774, which is longer than the specified 600
Created a chunk of size 954, which is longer than the specified 600
Created a chunk of size 922, which is longer than the specified 600
Created a chunk of size 1168, which is longer than the specified 600
Created a chunk of size 821, which is longer than the specified 600
Created a chunk of size 700, which is longer than the specified 600
Created a chunk of size 745, which is longer than the specified 600
Created a chunk of size 735, which is longer than the specified 600
Created a chunk of size 1110, which is longer than the specified 600
Created a chunk of size 991, which is longer than the specified 600
Created a chunk of size 990, which is longer than the specified 600
Created a chunk of size 1182, which is longer than the specified 600
Created a chunk of size 1491, which is longer than the specified 600
Created a chunk of size 1401, which is longe

[Document(metadata={'source': './files/chapter_one.txt'}, page_content='Part 1, Chapter 1\nPart One\n1 It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.'),
 Document(metadata={'source': './files/chapter_one.txt'}, page_content='The hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the ec

## Tiktoken

In [26]:
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

## Vector Store

In [27]:
from langchain_openai import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

# embedder.embed_query("Hi")
embedder.embed_documents(["Hi", "Hello"])


[[-0.03629858046770096,
  -0.007224537897855043,
  -0.03371885418891907,
  -0.02866363152861595,
  -0.02686564065515995,
  0.03460482135415077,
  -0.012318846769630909,
  -0.007752209436148405,
  0.0019380523590371013,
  -0.0027018729597330093,
  0.024781012907624245,
  -0.002477124100551009,
  -0.00573272630572319,
  -0.002905449829995632,
  0.006677323020994663,
  -0.00303248199634254,
  0.033849142491817474,
  -0.001503212028183043,
  0.02109382674098015,
  -0.008996471762657166,
  -0.02171921543776989,
  0.01038405206054449,
  0.006244111340492964,
  0.007081219926476479,
  -0.012312332168221474,
  0.0008998099947348237,
  0.005876044277101755,
  -0.009888952597975731,
  -0.0030731973238289356,
  -0.024572549387812614,
  0.010742347687482834,
  -0.01381065882742405,
  -0.024429231882095337,
  -0.01411032397300005,
  0.0024347801227122545,
  -0.018878910690546036,
  0.0005618723225779831,
  -0.011270018294453621,
  0.018110202625393867,
  -0.009967125952243805,
  0.01302892342209816

In [33]:
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader('./files/chapter_one.txt')

docs = loader.load_and_split(text_splitter=splitter)

vectorstore = FAISS.from_documents(docs, embedder)


In [34]:
vectorstore.similarity_search("where does winston live?")

[Document(id='c384ed39-9ffc-4f90-b0c4-f9079110c503', metadata={'source': './files/chapter_one.txt'}, page_content="The Ministry of Love was the really frightening one. There were no windows in it at all. Winston had never been inside the Ministry of Love, nor within half a kilometre of it. It was a place impossible to enter except on official business, and then only by penetrating through a maze of barbed-wire entanglements, steel doors, and hidden machine-gun nests. Even the streets leading up to its outer barriers were roamed by gorilla-faced guards in black uniforms, armed with jointed truncheons.\nWinston turned round abruptly. He had set his features into the expression of quiet optimism which it was advisable to wear when facing the telescreen. He crossed the room into the tiny kitchen. By leaving the Ministry at this time of day he had sacrificed his lunch in the canteen, and he was aware that there was no food in the kitchen except a hunk of dark-coloured bread which had got to

In [38]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(embedder, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embedder)

## Langsmith

In [39]:
from langsmith import utils

utils.tracing_is_enabled()

True

## RetrievalQA

In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(),
    
)

chain.run("Describe Victory Mansions.")

/Users/davidminjae/.local/share/virtualenvs/fullstack-gpt-Wpua4Cas/lib/python3.11/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


"Victory Mansions is a building with a hallway that smells of boiled cabbage and old rag mats. It has a colored poster of a man's face, about forty-five years old, with a heavy black mustache and ruggedly handsome features. The building has seven flights of stairs, with a poster of an enormous face on each landing that seems to follow you as you move. Inside the flat, there is a telescreen that cannot be completely shut off, and a fruity voice reading out figures related to pig-iron production."

## Stuff LCEL Chain

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human", "{question}")
])

chain = {"context":retriever, "question":RunnablePassthrough()} | prompt | llm 

chain.invoke("Describe Victory Mansions")

AIMessage(content='Victory Mansions is a building where Winston Smith resides. It has glass doors that let in gritty dust, and the hallway smells of boiled cabbage and old rag mats. The flat is seven flights up, and the building has a faulty lift due to the electric current being cut off during daylight hours as part of an economy drive. Inside the flat, there is a telescreen that cannot be completely shut off, and a poster with the caption "BIG BROTHER IS WATCHING YOU" hangs on the wall.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 2058, 'total_tokens': 2163, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-16765551-9784-4e3d-b4b7-00187b500fe8-0'

## Map Reduce LCEL Chain

In [44]:
from langchain.schema.runnable import RunnableLambda

map_doc_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim.
------
{context}
""",
    ),
    ("human", "{question}"),
])

map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke({
            "context": doc.page_content, 
            "question": question
        }).content for doc in documents
    )

map_chain = {"documents": retriever, "question": RunnablePassthrough()} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """Given the following extracted parts of a long document and a question, create a final answer. If you don't know the answer, just say that you don't know. Don't try to make up an answer.
------
{context}
""",
    ),
    ("human", "{question}"),
])

chain = {"question":RunnablePassthrough(), "context": map_chain} | final_prompt | llm
chain.invoke("Where does Winston go to work?")

AIMessage(content='Winston goes to work at the Ministry of Truth.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 154, 'total_tokens': 166, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a1731e1a-5a1f-49d7-b212-9a463154cd20-0', usage_metadata={'input_tokens': 154, 'output_tokens': 12, 'total_tokens': 166, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Function Calling

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

def get_weather(lon, lat):
    print("call an api...")

function = {
    "name": "get_weather",
    "description": "function that takes longitude and latitude to find the weather of a place",
    "parameters": {
        "type": "object",
        "properties": {
            "lon": {
                "type": "string",
                "description": "longitude of the place"
            },
            "lat": {
                "type": "string",
                "description": "latitude of the place"
            },
        },
    },
    "required": ["lon", "lat"],
}

llm = ChatOpenAI(
    temperature=0.1,
).bind(
    function_call="auto",
    functions=[
        function
    ]
)

prompt = PromptTemplate.from_template("Who is the weather in {city}")

chain = prompt | llm

response = chain.invoke({
    "city": "rome"
})

response = response.additional_kwargs["function_call"]["arguments"]

In [9]:
import json

r = json.loads(response)

get_weather(r['lon'], r['lat'])

call an api...
